<a href="https://colab.research.google.com/github/luiscaballeroramos/DeCoder/blob/EC3_1_8_DesignOfJoints/DeCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preliminary Tasks

Install and import PyPDF2 library to open and read the PDF file containing the code

In [1]:
!pip install PyPDF2
import PyPDF2

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=3878991a050f75bdb3b659302a1138c1cf35cc27d232eeb74a8304a5662548f0
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


Clone the GitHub repository containing the PDF file with code

In [2]:
!git clone -b EC3_1_8_DesignOfJoints https://github.com/luiscaballeroramos/DeCoder.git

Cloning into 'DeCoder'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 54 (delta 28), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), done.


Import libraries used in this tool

In [3]:
import numpy as np
import regex as re

# 1. Read PDF

Read the .pdf file with PyPF2 library

In [4]:
pdfFile=open('DeCoder/EC 1993.1.8.2005-1.pdf','rb')
pdfReader=PyPDF2.PdfFileReader(pdfFile)

In [5]:
numPages=pdfReader.numPages
print(numPages)

135


Get the pages content as dictionary (pages) with:
* key = page number (pageNum)
* value = page content as string (pageContent)



In [6]:
pages={}
pageContent=''
for pageNum in range(0,numPages):
  pdfPage=pdfReader.getPage(pageNum)
  pageContent=pdfPage.extractText()
  pages[pageNum]=pageContent
  pass

# 2. Table Of Contents (TOC)
2.0. TOC Definition (settings)

2.1. TOC Location (pages)

2.2. TOC Contents (toc's)

# 2.0. TOC Definition

Set TOC identifiers, keywords that could be used to refering TOC

In [43]:
TOCidentifiers=['contents','Contents',
                 'table of contents','Table of contents','Table Of Contents']

Description of toc elements by regex expressions 

In [44]:
refRegex='\d(?:\.+\d*)+'
namRegex='[\s\w]*'
sepRegex='\.{2,}'
numRegex='\d+'

In [69]:
class TOC():
  def __init__(self,refRegex,namRegex,sepRegex,numRegex):
    self.refRegex=refRegex
    self.namRegex=namRegex
    self.sepRegex=sepRegex
    self.numRegex=numRegex
    self.tocRegex=refRegex+'\s*'+namRegex+'\s*'+sepRegex+'\s*'+numRegex
    self.tocList=[]
    pass
  def addToc(self,toc):
    self.tocList.append(toc)
    pass
  pass
TOC_EC3=TOC(refRegex,namRegex,sepRegex,numRegex)

In [68]:
class toc():
  def __init__(self,ref,name,begin=None,end=None,level=None):
    self.ref=ref
    self.name=name
    self.begin=begin
    self.end=end
    self.level=level
    pass
  pass

In [62]:
class PDFlocation():
  def __init__(self,page,begin,end=0):
    self.page=page
    self.begin=begin
    if end==0:
      end=begin+1
      pass
    self.end=end
    pass
  pass

# 2.1. TOC Location

TOC identifiers apearances in each document page 

In [47]:
TOCcoincidences={}
TOCtotalCoincidences={}
for pageNum, pageContent in pages.items():
  aux=[]
  for TOCidentifier in TOCidentifiers:
    aux.append(pageContent.count(TOCidentifier))
    pass
  TOCcoincidences[pageNum]=aux
  TOCtotalCoincidences[pageNum]=sum(aux)
  pass

Total of TOC identifiers apearances in PDF document

In [48]:
sumTOCTotalCoincidences=sum(TOC_totalCoincidences.values())
print('Number of TOC indentifiers: {0}'.format(sumTOCTotalCoincidences))

Number of TOC indentifiers: 1


Max TOC identifiers apearances in one page and the page where this happens

In [49]:
maxTOCTotalCoincidences=max(TOCtotalCoincidences.values())
maxTOCTotalCoincidencesPages = [key  for (key, value) in TOC_totalCoincidences.items() if value == maxTotalCoincidences]
print('Number of Max TOC indentifiers in One Page: {0}'.format(maxTotalCoincidences))
print('Pages with Max TOC indentifiers in One Page: {0}'.format(maxTotalCoincidencesPages))

Number of Max TOC indentifiers in One Page: 1
Pages with Max TOC indentifiers in One Page: [3]


Pages where toc pattern (Regex) is detected

In [58]:
tocCoincidences={}
for pageNum, pageContent in pages.items():
  tocCoincidences[pageNum]=len(re.findall(TOC_EC3.tocRegex,pages[pageNum]))
  pass

In [57]:
noNulltocCoincidencesPages = [key  for (key, value) in tocCoincidences.items() if value != 0]
print('Pages with toc Regex: {0}'.format(noNulltocCoincidencesPages))

Pages with toc Regex: [3, 4, 5]


In [59]:
TOCpages=noNulltocCoincidencesPages

# 2.2. TOC Contents

In [71]:
pattern = re.compile(TOC_EC3.tocRegex)
for page in TOCpages:
  for match in pattern.finditer(pages[page]):
    coincidence=match.group()
    ref=re.search(TOC_EC3.refRegex,coincidence)
    nam=re.search(TOC_EC3.namRegex,coincidence)
    beginLocation=PDFlocation(page,match.start())
    endLocation=PDFlocation(page,match.end())
    #print(match.start(), match.end(), match.group())
    TOC_EC3.addToc(toc(ref,nam,beginLocation,endLocation))
    pass
  pass
for toc in TOC_EC3.tocList:
  print(toc.ref,toc.name,toc.begin,toc.end,toc.level)
  pass
#print(pages[3][521:648])

TypeError: ignored

**2.1. Reference**

In [ ]:
tocList=re.findall('\s\d(?:\.+\d*)+\s',pages[3])
print(tocList)
print(len(tocList))

[' 1.3 ', ' 1.5 ', ' 2.1 ', ' 2.2 ', ' 2.3 ', ' 2.4 ', ' 2.5 ', ' 2.6 ', ' 2.7 ', ' 3.1.1 ', ' 3.2 ', ' 3.4 ', ' 3.4. ', ' 3.4.2 ', ' 3.5 ', ' 3.6 ', ' 3.6.1 ', ' 3.6.2 ', ' 3.7 ', ' 3.8 ', ' 3.9 ', ' 8.8 ', ' 3.9. ', ' 3.9.2 ', ' 3.9.3 ', ' 3.10 ', ' 3.10.1 ', ' 3.10.2 ', ' 3.10.3 ', ' 3.10.4 ', ' 3.1 ', ' 3.12 ', ' 3. ', ' 3.1 ', ' 3.13.2 ', ' 4.1 ', ' 4.2 ', ' 4.3 ', ' 4.3.2 ', ' 4.3.3 ', ' 4.3.4 ', ' 4.3.5 ']
42


**2.2. Name**


In [ ]:
tocList=re.findall('[\s\w]*',pages[3])
print(tocList)
print(len(tocList))

['BS EN 1993', '', '1', '', '8', '', '2005 EN 1993', '', '1', '', '8', '', '2005 ', '', 'E', '', ' Contents Page Introduction ', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ' 8 1', '', '', ' Scope ', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

**2.3 Separator**

In [ ]:
tocList=re.findall('\.{2,}',pages[3])
print(tocList)
print(len(tocList))

['.............................................................................................................................................', '.................................................................................................................................................', '.........................................................................................................................', '....................................................................', '.......................................................................................................................', '............................................................................................................................................', '.........................................................................................................................................', '...................................................................................................

# Close PDF

In [ ]:
pdfFile.close()